In [ ]:
# Import package for Data Extraction, Loading & Transformation
import pandas as pd
import requests

# Import dotenv package for setting environment variables 
from dotenv import load_dotenv

# Import os package
import os

# Set environment variables from the .env in the local environment
load_dotenv()

# Retrieve API key and store as Python variable
api_key = os.getenv('API_KEY')
type(api_key)

In [ ]:
# ETFs and Ticker Data for Sectoral Performance Analysis

ticker_name_dict = {'XIU':['iShares S&P/TSX 60 Index ETF','TSX_60'],
                    'XST':['iShares S&P/TSX Capped Consumer Staples Index ETF','Consumer Staples'],
                    'XHC':['iShares Global Healthcare Index ETF','Health'],
                    'XRE':['iShares S&P/TSX Capped REIT Index ETF','Real Estate'],
                    'XUT':['iShares S&P/TSX Capped Utilities Index ETF','Utilities'],
                    'CEW':['iShares Equal Weight Banc & Lifeco ETF','Financial Services'],
                    'TRVL':['Harvest Travel & Leisure Index ETF','Recreation & Leisure'],
                    'GLD':['SPDR Gold Shares','Gold ETF']}

ticker_df = pd.DataFrame.from_dict(ticker_name_dict,orient='index',
                                   columns=['ETF','Sector'])

ticker_df                                       

In [ ]:
# Creating New Directory to Save Data

os.makedirs('./Output', exist_ok = True)
ticker_df.to_csv('./Output/ticker.csv')

In [ ]:
# Using Function to extract data using APIs and save as csvfiles

etf_list = ticker_df.index.to_list() # Creating a List of ETF Tickers

def extract_api_data(data_list):
    df_list = []
    
    for item in data_list:
        url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={item}.TRT&outputsize=full&apikey={api_key}&datatype=csv'
        r = requests.get(url)
        df = pd.read_csv(url)
        df['timestamp'] = df['timestamp'].str[:4]
        item_df = df.rename(columns={'close': f'{item}','timestamp':'Year'})
        item_df = item_df.groupby('Year').mean()
        df_list = df_list.append(item_df)
    
    raw_df = pd.concat(df_list)
    return(raw_df)

raw_etf_df = extract_api_data(etf_list)

combined_etf_df = raw_etf_df.drop(columns=['open','high','low','volume'],axis=1)

combined_etf_df.to_csv('./Output/etf_annual_data.csv')

display(combined_etf_df.head())
display(combined_etf_df.tail())


In [ ]:
for item in etf_list:
        url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={item}.TRT&outputsize=full&apikey={api_key}&datatype=csv'
        r = requests.get(url)
        df = pd.read_csv(url)
        display(df.head())
